# Twitter Automation Scanner - Dependencies Graph

## Step 1 - Install required dependencies

- [networkx](https://networkx.org) - NetworkX is a Python package for the creation, manipulation, and study of the structure, dynamics, and functions of complex networks.
- [matplotlib](https://matplotlib.org) - Matplotlib is a comprehensive library for creating static, animated, and interactive visualizations in Python. Matplotlib makes easy things easy and hard things possible.

In [ ]:
pip install networkx matplotlib

## Step 2 - Load source JSON file

In [ ]:
import json
import os

folder_walk = os.walk('sourcedata/dbdump')
first_file_in_folder = next(folder_walk)[2][0]

report_file = open(f'{ os.getcwd() }/sourcedata/dbdump/{first_file_in_folder}')
user_data = json.load(report_file)

users = []

for user in user_data:
    users.append((user['_id'],{'followers': user['followers'], 'following': user['following'], 'id': user['_id']}))

## Step 3 - Draw dependencies graph

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import networkx as nx

G = nx.DiGraph()

usernames = list()

# Set printed graph size
plt.rcParams["figure.figsize"] = (12, 6)

# Get a list of usernames
for user in users:
    usernames.append(user[0])

# Fill in graph
for user in users:
    username = user[0]
    
    for follower in user[1]['followers']:
        if follower in usernames and G.has_edge(follower, username) == False:
            G.add_edge(follower, username)
    for follower in user[1]['following']:
        if follower in usernames and G.has_edge(username, follower) == False:
            G.add_edge(username, follower)         
            
pos = nx.spring_layout(G)

# Set node sizes
node_sizes = list()

for node in list(G.nodes):
    node_sizes.append( 100 + 100 * (len(list(G.edges(node)))))
    print( node, 100 + 100 * (len(list(G.edges(node)))) )
    
# Draw nodes
nx.draw_networkx_nodes(G, pos, node_size=node_sizes, node_color="orange")

# Draw edges
nx.draw_networkx_edges(
    G,
    pos,
    width=1,
    arrowstyle="-|>",
    arrowsize=15,
    node_size=list(map(lambda n: n + 200, node_sizes)),
    edge_color="gray",
    arrows=True
)

# Draw node labels
nx.draw_networkx_labels(G, pos, font_size=8)

plt.axis("off")
plt.show()